# CH-07 年報問答機器人

## 7-2 取得年報

### 1️⃣  匯入套件

In [1]:
import requests
from bs4 import BeautifulSoup

### 2️⃣ 建立函式-取得年報

In [2]:
def annual_report(id,y):
  url = 'https://doc.twse.com.tw/server-java/t57sb01'
  # 建立 POST 請求的表單
  data = {
      "id":"",
      "key":"",
      "step":"1",
      "co_id":id,
      "year":y,
      "seamon":"",
      "mtype":'F',
      "dtype":'F04'
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data)
    # 取得回應後擷取檔案名稱
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a').text
    print(link1)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 建立第二個 POST 請求的表單
  data2 = {
      'step':'9',
      'kind':'F',
      'co_id':id,
      'filename':link1 # 檔案名稱
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data2)
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a')
    # 取得 PDF 連結
    link2 = link1.get('href')
    print(link2)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 發送 GET 請求
  try:
    response = requests.get('https://doc.twse.com.tw' + link2)
    # 取得 PDF 資料
    with open(y + '_' + id + '.pdf', 'wb') as file:
        file.write(response.content)
    print('OK')
  except Exception as e:
    print(f"發生{e}錯誤")

### 3️⃣ 呼叫函式

In [3]:
annual_report('2330','113')

2023_2330_20240604F04.pdf
/pdf/2023_2330_20240604F04_20250216_065514.pdf
OK


## 7-3 年報問答

langchain 以更新至新版寫法，舊版可參考：
https://colab.research.google.com/drive/16x0mUitJjH0PZx7kk2Kew2MSlkB9G_lA

###4️⃣ 安裝相關套件

In [4]:
!pip install pydantic
!pip install pdfplumber langchain langchain_community
!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00


###  5️⃣ 匯入相關套件

In [5]:
import os
import getpass
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

### 6️⃣ 設定環境變數和建立 Google Gemini 模型

In [6]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

llm_model = ChatGoogleGenerativeAI(
    api_key = GEMINI_API_KEY,
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

### 7️⃣ 建立函式-建立向量資料庫

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

def build_vector_db(file_path, size, overlap):
    loader = PDFPlumberLoader(file_path)
    doc = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=overlap)
    docs = text_splitter.split_documents(doc)
    db = InMemoryVectorStore.from_documents(docs, embeddings)
    return db

### 8️⃣  呼叫函式

In [ ]:
db = build_vector_db('/content/113_2330.pdf', 500, 50)

### 9️⃣ 查詢相關資料

In [ ]:
similarity_docs = db.similarity_search("研發的產品", k=5)
for i in similarity_docs:
    print(i.page_content)
    print('_________')

### 🔟  匯入問答相關套件

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

### 1️⃣1️⃣  建立函式-問答程式

In [ ]:
# 提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一個根據年報資料與上下文作回答的助手,"
     "如果有明確數據或技術(產品)名稱可以用數據或名稱回答,"
     "回答以繁體中文為主。"
     "{context}"),
    ("human","{question}")])

# 建立問答函式
def question_and_answer(question):
    retrievalQA = RetrievalQA.from_llm(
        llm=llm_model,
        prompt=prompt,
        return_source_documents=True,
        retriever=db.as_retriever(
        search_kwargs={'k':8}))
    answer = retrievalQA.invoke(question)
    return answer

### 1️⃣2️⃣ 建立迴圈進行問答

In [ ]:
while True:
    question = input("輸入問題:")
    if not question.strip():
        break
    result = question_and_answer(question)
    print(result['result'])
    print('_________')
    #print(result["source_documents"])

## 7-4 年報總結與分析

### 1️⃣3️⃣ 回答結果及原始資料

In [ ]:
from langchain.chains.summarize import load_summarize_chain

### 1️⃣4️⃣ 總結原始資料

In [ ]:
# 建立關鍵字串列
key_word = ['正在開發的產品及銷售狀況',
            '市場策略的調整或變化',
            '公司預期未來展望',
            '總營收、稅前淨利的成長或變動分析',
            '國際競爭以及海外市場銷售情形']

data_list = []
for word in key_word:
    data = db.similarity_search(word, k=3)
    # 整合 Document 串列
    data_list += data

# 建立提示訊息串列
prompt_template = [("system","你的任務是對年報資訊進行摘要總結。"
                    "以下為提供的年報資訊：{text},"
                    "請給我重點數據, 如銷售增長情形、營收變化、開發項目等,"
                    "最後請使用繁體中文輸出報告")]
prompt = ChatPromptTemplate.from_messages(messages=prompt_template)

### 1️⃣5️⃣  呼叫函式

In [ ]:
refine_chain = load_summarize_chain(llm=llm_model,
                                    chain_type='stuff',
                                    prompt=prompt)
summary_refine = refine_chain.invoke({"input_documents": data_list})
print(summary_refine['output_text'])

### 1️⃣6️⃣  提取關鍵字
使用 MMR 搜尋方法

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

word_prompt = PromptTemplate.from_template(
     "從{input}聯想出4個與年報分析有關的重要關鍵字,"\
     "請確保回答具有具有關聯性、多樣性和變化性。 \n "
     "僅回覆關鍵字, 並以半形逗號與空格來分隔。不要加入其他內容")

word_chain = word_prompt | llm_model | CommaSeparatedListOutputParser()
print(word_chain.invoke({"input": "公司的營運狀況如何?"}))

### 1️⃣7️⃣ 設定 AI 角色讓其分析報告

In [ ]:
data_prompt = PromptTemplate.from_template(
    "你現在是一位專業的股票分析師,"
    "會以詳細、嚴謹的角度針對 {key_words} 進行年報分析,"
    "請提及關於營收、是否成長以及利潤等重要數字,"
    "最後生成一份專業的趨勢分析報告。"
    "以下為年報資料：{data_content}")

data_chain = data_prompt | llm_model | StrOutputParser()

### 1️⃣8️⃣ 整合函式

In [ ]:
def analyze_chain(input):
    # 搜尋「問題」的相關資料
    data = db.max_marginal_relevance_search(input, fetch_k=5, k=2)

    # 第一個 Chain 元件, 建立「關鍵字」串列
    word_list = word_chain.invoke({"input": input})

    # 搜尋「關鍵字」的相關資料
    for word in word_list:
      data += db.max_marginal_relevance_search(word, fetch_k=5, k=2)
    word_list.append(input)

    # 第二個 Chain 元件, 生成分析報告
    result = data_chain.invoke({'key_words':word_list,'data_content':data})

    return result

### 1️⃣9️⃣ 呼叫函式

In [ ]:
input = '公司的營收狀況如何？'
analyze_report = analyze_chain(input)
print(analyze_report)